In [2]:
import glob 
import pandas as pd 
import numpy as np 
import os 
from utils import * 
import subprocess
import itertools

%load_ext autoreload
%autoreload 2

In [3]:
sample_metadata_df = pd.read_csv('sample_metadata.csv', index_col=0)
sample_metadata_df['metat'] = sample_metadata_df.sample_id.str.contains('metat')

ggkbase_name_to_year_map = sample_metadata_df.set_index('ggkbase_name').year.to_dict()
sample_id_to_year_map = sample_metadata_df.set_index('sample_id').year.to_dict()
ggkbase_name_to_sample_id_map = sample_metadata_df.set_index('ggkbase_name').sample_id.to_dict()
sample_id_to_ggkbase_name_map = sample_metadata_df.set_index('sample_id').ggkbase_name.to_dict()

sample_path_template = '/groups/banfield/sequences/{year}/{ggkbase_name}/raw.d/{ggkbase_name}_trim_clean.{paired_end}.fastq.gz'
ref_genome_dir = '../data/'
ref_genome_paths = [f'../data/data/{file_name}.fn' for file_name in id_to_ggkbase_name_map.keys()]

sample_ids = sample_metadata_df.sample_id.unique()
genome_ids = list(id_to_ggkbase_name_map.keys())

In [4]:
def clean_fasta_file(path:str):
    '''Remove extra information from the FASTA file headers.'''
    subprocess.run(f"sed -i 's/ .*//' {path}", shell=True, check=True)

for path in ref_genome_paths:
    clean_fasta_file(path)

In [5]:
coverm_sample_paths = '' 
for row in sample_metadata_df[~sample_metadata_df.metat].itertuples():
    coverm_sample_paths += sample_path_template.format(ggkbase_name=row.ggkbase_name, paired_end='PE.1', year=row.year) + ' '
    coverm_sample_paths += sample_path_template.format(ggkbase_name=row.ggkbase_name, paired_end='PE.2', year=row.year) + ' '
coverm_sample_paths = coverm_sample_paths.strip()

coverm_fields = 'mean trimmed_mean covered_bases variance count rpkm tpm'

with open('../scripts/coverm_mapping.sh', 'w') as f:
    for genome_id in id_to_ggkbase_name_map.keys():
        ref_genome_path = os.path.join(ref_genome_dir, f'{genome_id}.fn')

        if 'metat' in genome_id: # Don't do this for the transcripts. 
            continue 

        output_file_name = f'{genome_id}.tsv'.lower()
        output_path = f'/home/philippar/data/coverm/{output_file_name}'
        cmd = f'coverm contig -c {coverm_sample_paths} -r {ref_genome_path} --min-read-percent-identity 97 --min-read-aligned-percent 80 --trim-min 5 --trim-max 95 -m {coverm_fields} -t 20 -o {output_path}'
        f.write(f'sbatch --wrap "{cmd}"\n')

In [6]:
# with open('../scripts/bbduk_library_sizes.sh', 'w') as f:
#     for name, sample_id in sample_id_map.items():
#             paired_ends_1_path = sample_path_template.format(paired_end='PE.1', sample_id=sample_id, year=sample_year_map.get(sample_id, 2025))
#             paired_ends_2_path = sample_path_template.format(paired_end='PE.2', sample_id=sample_id, year=sample_year_map.get(sample_id, 2025))
#             output_path = f'/home/philippar/data/bbduk/{name}.txt'
#             cmd = f'bbduk.sh in={paired_ends_1_path} in2={paired_ends_2_path} out=/dev/null stats={output_path} threads=64'
#             f.write(f'sbatch --wrap "{cmd}"\n')

In [7]:
def bbmap_get_command(sample_id:str, genome_id:str, output_dir:str='../data/metat/', input_dir='../data/'):
    ref_genome_path = os.path.join(input_dir, f'{genome_id}.fn')

    input_path_1 = sample_path_template.format(ggkbase_name=sample_id_to_ggkbase_name_map[sample_id], paired_end='PE.1', year=sample_id_to_year_map[sample_id])
    input_path_2 = sample_path_template.format(ggkbase_name=sample_id_to_ggkbase_name_map[sample_id], paired_end='PE.2', year=sample_id_to_year_map[sample_id])

    output_dir = os.path.join(output_dir, genome_id)
    output_path = os.path.join(output_dir, f'{sample_id}.bam')

    params = 'pigz=t unpigz=t ambiguous=random minid=0.96 idfilter=0.97 threads=64 out=stdout.sam editfilter=5 out=stdout.sam'
    cmd = f'bbmap.sh {params} in1={input_path_1} in2={input_path_2} ref={ref_genome_path} nodisk | shrinksam | sambam > {output_path}'
    return cmd, output_path


output_dir = '../data/metat'

with open('../scripts/bbmap_mapping.sh', 'w') as f:
    for genome_id in genome_ids:
        f.write(f'mkdir -p {os.path.join(output_dir, genome_id)}\n') # Make sure the output directory exists.
        for sample_id in sample_ids:
            if 'metat' in sample_id:
                cmd, output_path = bbmap_get_command(sample_id, genome_id, output_dir=output_dir)
                if not os.path.exists(output_path):
                    f.write(f'sbatch --wrap "{cmd}"\n')



In [25]:
def featurecounts_get_command(bam_path:str, ref_path:str=None, output_dir:str=None):
    output_file_name = os.path.basename(bam_path).replace('.bam', '')
    output_file_name += '_read_counts'
    output_path = os.path.join(output_dir, output_file_name)
    return f'featureCounts -p -T 64 -g ID -t CDS -a {ref_path} -s 2 -o {output_path} {bam_path}' 

data_dir = '~/data/'
output_dir = '~/data/metat'

with open('../scripts/featurecounts_counting.sh', 'w') as f:
    for genome_id in genome_ids:
        dir_ = os.path.join(output_dir, genome_id)
        for sample_id in sample_ids:
            if 'metat' not in sample_id:
                continue
            bam_path = os.path.join(dir_, f'{sample_id}.bam')
            ref_path = os.path.join(data_dir, f'{genome_id}.gff')
            cmd = featurecounts_get_command(bam_path, ref_path, output_dir=dir_)
            
            f.write(cmd + '\n')

In [22]:
sample_ids

array(['ck_bottom_2025', 'ck_middle_2025', 'ck_top_2025', 'n_bottom_2025',
       'n_middle_2025', 'n_top_2025', 'n_middle_2024', 'n_top_2024',
       'n_bottom_2024', 'ck_bottom_2024', '90cm_2020', '100cm_2020',
       '100cm_2021_1', '100cm_2021_2', '80cm_2023', '40cm_2020',
       '100cm_2022', '90cm_2023', '60cm_2023', 'ck_bottom_2025_metat',
       'ck_middle_2025_metat', 'ck_top_2025_metat', 'n_top_2025_metat',
       'n_bottom_2025_metat', 'n_middle_2025_metat',
       'ck_bottom_2024_metat', 'n_top_2024_metat', 'n_bottom_2024_metat',
       'n_middle_2024_metat'], dtype=object)

In [23]:

def interproscan_get_command(input_path:str, output_dir:str='~/data/interproscan/'):
    id_ = os.path.basename(input_path).split('.')[0]
    output_path = os.path.join(output_dir, f'{id_}.tsv')
    cmd = f'interproscan.sh -i {input_path} -o {output_path} -f tsv'
    return cmd

with open('../scripts/interproscan_annotation.sh', 'w') as f:
    for genome_id in genome_ids:
        input_path = f'~/data/{genome_id}.fa'
        cmd = interproscan_get_command(input_path)
        f.write(cmd + '\n')

In [14]:
def kofamscan_get_command(input_path:str, output_dir:str='~/data/kofamscan/'):
    id_ = os.path.basename(input_path).split('.')[0]
    output_path = os.path.join(output_dir, f'{id_}.tsv')
    profiles_path = '/shared/db/kegg/kofam/latest/profiles'
    ko_list_path = '/shared/db/kegg/kofam/latest/metadata/ko_list'
    cmd = f'/shared/software/kofamscan/latest/exec_annotation {input_path} -o {output_path} -f detail-tsv -E 0.1 --cpu 16 -k {ko_list_path} -p {profiles_path}'
    cmd = f'sbatch --wrap "{cmd}"'
    return cmd

with open('../scripts/kofamscan_annotation.sh', 'w') as f:
    for genome_id in genome_ids:
        input_path = f'~/data/{genome_id}.fa'
        cmd = kofamscan_get_command(input_path)
        f.write(cmd + '\n')